In [1]:
# # encoding=utf8
# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

In [2]:
import pandas as pd
import os
from time import time, ctime

import concurrent.futures

from lxml import etree
import json
import re

import ast

In [3]:
# path='technosystem.html'

folder='bidder_pages/'
cont = os.listdir(folder)
cont = [i for i in cont if i[-4:]=='html']
len(cont)

68229

In [4]:
def flatten_dict(data, layers=1, drop_deeper=False):

        for _ in range(layers):
            data = [(k, v) if not isinstance(v, dict) else [(k + '.' + k2, v2) for k2, v2 in v.items()] for k, v in
                    data.items()]
            data = [item for sublist in data for item in sublist if isinstance(sublist, list)] + [y for y in data if
                                                                                                  not isinstance(y,
                                                                                                                 list)]
            data = dict(data)

        if drop_deeper:
            data = {k: v for k, v in data.items() if not isinstance(v, dict) or isinstance(v, list)}

        return data

In [5]:
from collections import defaultdict
iniDict = defaultdict(lambda: None)

In [106]:
# df = pd.read_csv('boss1.csv').set_index('name')
# for i in list(df.index):
#     iniDict[i]
# len(iniDict)

In [14]:
initial = pd.DataFrame(columns=list(df.index))
initial.shape

(0, 73)

In [15]:
initial

,about.support_status,about.test_user,viewer,about.avatar,about.tagline,userId,about.status,about.location,timeOffset,isSupportAgent,...,skills,about.corporate_users,about.user_id,isOwner,about.first_name,about.primary_currency,about.limited_account,about.address,about.employer_reputation,about.badges


In [16]:
cont[0]

'Ukrabel.html'

In [17]:
pd.DataFrame([[1,2,3]], index=['ac'])

,0,1,2
ac,1,2,3


In [18]:
pd.DataFrame([[1,2,3]], index=['ac']).set_index

<bound method DataFrame.set_index of     0  1  2
ac  1  2  3>

In [19]:
pd.DataFrame(initial, index=['abc'])

,about.support_status,about.test_user,viewer,about.avatar,about.tagline,userId,about.status,about.location,timeOffset,isSupportAgent,...,skills,about.corporate_users,about.user_id,isOwner,about.first_name,about.primary_currency,about.limited_account,about.address,about.employer_reputation,about.badges
abc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# aaa = [pd.DataFrame(initial, index=[3]), pd.DataFrame(initial, index=[6]), pd.DataFrame(initial, index=[8])]
# len(aaa)

In [21]:
# for i in aaa:
#     initial=initial.append(i)
# initial

In [6]:

def get_columns(path):
    from time import ctime
    global failed_names, collect, n
    
    n+=1
    if n==10000 or n==0:
        print ctime(), ': processed', n
    get=[]
    
    with open(folder+path,'r') as fl:
        text=fl.read()
#         tree=etree.HTML(text)
    try:
#         script = tree.xpath('//script')[-2]
        data = re.search(r"__export\('profileServerData',(.*?)\n", text)        
        jsondata=data.group(1)[:-2]
        
        json_dict = json.loads(jsondata)
        flat = flatten_dict(json_dict, layers=1)
        for i in flat:
            iniDict[i]=str(flat[i])
        collect.append(pd.DataFrame(iniDict, index=[path]))
            
#         return pd.DataFrame(iniDict, index=[0])
#         names = flat.keys()
#         return flat
    except Exception as e:
        print e
        print 'X',
        failed_names.append(path)
        collect.append(pd.DataFrame(index=[path]))
        

    

1533

### multi

In [7]:
def batch(allnames):
    global collect
    start=time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        executor.map(get_columns, allnames)
    past='{:.2f}'.format(time()-start)
    now=ctime()[4:]

    print 'profiles ',len(allnames),'  used ',past,'s', now
    return past


### multi starts

In [8]:
# initialize
n=0
initial = pd.DataFrame(columns=list(df.index))
collect=[]
failed_names=[]


# start
eclipse = batch(cont[:100])

for i in collect:
    initial=initial.append(i)
    
eclipse

NameError: name 'df' is not defined

In [91]:
len(failed_names)

208

In [92]:
len(failed_names)+initial.shape[0]

1208

In [93]:
start=initial.copy()

In [94]:
print initial.shape

In [95]:
delete=[]
for i in start:
#     print i
#     print start[i].value_counts().head(5)
#     print 
#     print 
    try:
        if start[i].value_counts(dropna=False).iloc[:2].sum()==start.shape[0]:
            delete.append(i)
    except:
        print i
#         print i
len(delete)

35

In [96]:
start = start.drop(delete, axis=1)


In [99]:
list(start.columns)

['about.avatar',
 'about.avatar_cdn',
 'about.avatar_large',
 'about.avatar_large_cdn',
 'about.chosen_role',
 'about.company',
 'about.corporate',
 'about.cover_image',
 'about.display_name',
 'about.hourly_rate',
 'about.id',
 'about.location',
 'about.preferred_freelancer',
 'about.primary_currency',
 'about.primary_language',
 'about.profile_description',
 'about.public_name',
 'about.recommendations',
 'about.registration_date',
 'about.registration_date_human',
 'about.reputation',
 'about.role',
 'about.status',
 'about.tagline',
 'about.timezone',
 'about.username',
 'about.username_sentence',
 'hourlyRate',
 'resume.certifications',
 'resume.education',
 'resume.exams',
 'resume.experience',
 'resume.publications',
 'resume.show',
 'skills',
 'timeOffset',
 'userId',
 'username']

In [83]:
start = start.reset_index(drop=True)
start.to_csv('bidder1_JSON_0.csv',index=False)

In [77]:
print 'done'

(56272, 38)

In [ ]:
from time import ctime

In [ ]:
print ctime()

In [107]:
b = pd.read_csv('bidder1_JSON.csv')
b.shape

(1000, 38)

In [108]:
b

,about.avatar,about.avatar_cdn,about.avatar_large,about.avatar_large_cdn,about.chosen_role,about.company,about.corporate,about.cover_image,about.display_name,about.hourly_rate,...,resume.certifications,resume.education,resume.exams,resume.experience,resume.publications,resume.show,skills,timeOffset,userId,username
0,/ppic/165393/logo/1315741/profile_logo_1315741...,//cdn6.f-cdn.com/ppic/165393/logo/1315741/prof...,/ppic/264223/logo/1315741/1.jpg,//cdn2.f-cdn.com/ppic/264223/logo/1315741/1.jpg,1.0,EDC,None,{u'current_image': {u'url': u'//cdn2.f-cdn.com...,Ukrabel,7,...,[],"[{u'activities': None, u'start_year': u'2009',...",[],[],[],True,"[{u'usages': 0, u'seo_url': u'copywriting', u'...",180.0,1315741.0,ukrabel
1,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,1.0,Ixintro,None,{u'current_image': {u'url': u'//cdn2.f-cdn.com...,Ixintro,8,...,[],[],[],[],[],False,"[{u'usages': 0, u'seo_url': u'flash', u'isloca...",-240.0,1275180.0,ixintro
2,/ppic/2741669/logo/1495236/profile_logo_149523...,//cdn6.f-cdn.com/ppic/2741669/logo/1495236/pro...,/ppic/2741668/logo/1495236/will_work_for_a_whi...,//cdn3.f-cdn.com/ppic/2741668/logo/1495236/wil...,1.0,Home,None,{u'current_image': {u'url': u'//cdn2.f-cdn.com...,Sergey,10,...,[],"[{u'activities': None, u'start_year': u'2004',...",[],[],[],True,"[{u'usages': 0, u'seo_url': u'php', u'islocal'...",180.0,1495236.0,glukodrom
3,/ppic/103601/logo/295905/profile_logo_295905.jpg,//cdn2.f-cdn.com/ppic/103601/logo/295905/profi...,/ppic/12967/logo/295905/logo.jpg,//cdn6.f-cdn.com/ppic/12967/logo/295905/logo.jpg,1.0,INICSOL,None,{u'current_image': {u'url': u'//cdn6.f-cdn.com...,Inicsol,16,...,"[{u'award_date': u'2005', u'descr': u'Aptech C...","[{u'activities': None, u'start_year': u'2000',...","{u'2': {u'examEnabled': u'1', u'description': ...","[{u'year_interval': u'-2007', u'month_interval...","[{u'publisher': u'Spider Magzine', u'title': u...",True,"[{u'usages': 22, u'seo_url': u'php', u'islocal...",300.0,295905.0,observer
4,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,3.0,Noor International,None,{u'current_image': {u'url': u'//cdn2.f-cdn.com...,blugraphic,20,...,[],[],[],[],[],False,"[{u'usages': 0, u'seo_url': u'php', u'islocal'...",180.0,1355684.0,blugraphic
5,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,1.0,NaN,None,{u'current_image': {u'url': u'//cdn2.f-cdn.com...,RichardWeiss,12,...,[],[],[],[],[],False,"[{u'usages': 1, u'seo_url': u'articles', u'isl...",60.0,1450038.0,richardweiss
6,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,1.0,Creatus,None,{u'current_image': {u'url': u'//cdn2.f-cdn.com...,spaprikar,10,...,[],[],[],[],[],False,"[{u'usages': 0, u'seo_url': u'flash', u'isloca...",330.0,1469479.0,spaprikar
7,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,1.0,Raj Infotech,None,{u'current_image': {u'url': u'//cdn2.f-cdn.com...,p30patel,None,...,[],[],[],[],[],False,"[{u'usages': 0, u'seo_url': u'asp', u'islocal'...",-240.0,1388559.0,p30patel
8,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,/img/unknown.png,//cdn6.f-cdn.com/img/unknown.png,1.0,freelancer,None,{u'current_image': {u'url': u'//cdn2.f-cdn.com...,mcabezas,8,...,[],[{u'activities': u'Former President of the Com...,[],[],[],True,"[{u'usages': 0, u'seo_url': u'visual_basic', u...",-360.0,1520721.0,mcabezas
9,/ppic/103496799/logo/1570175/profile_logo_1570...,//cdn5.f-cdn.com/ppic/103496799/logo/1570175/p...,/ppic/103496797/logo/1570175/QRxqp/profile_log...,//cdn2.f-cdn.com/ppic/103496797/logo/1570175/Q...,3.0,NaN,None,{u'current_image': {u'url': u'//cdn2.f-cdn.com...,John Inso,30,...,[],[],[],[],[],False,"[{u'usages': 0, u'seo_url': u'php', u'islocal'...",480.0,1570175.0,jdbkrihn123
